In [43]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Logical;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

In [44]:
// Task 4 (12 points). f(x) = 1 if the graph edge coloring is triangle-free
// 
// Inputs:
//      1) The number of vertices in the graph "V" (V ≤ 6).
//      2) An array of E tuples of integers "edges", representing the edges of the graph (0 ≤ E ≤ V(V-1)/2).
//         Each tuple gives the indices of the start and the end vertices of the edge.
//         The vertices are indexed 0 through V - 1.
//         The graph is undirected, so the order of the start and the end vertices in the edge doesn't matter.
//      3) An array of E qubits "colorsRegister" that encodes the color assignments of the edges.
//         Each color will be 0 or 1 (stored in 1 qubit).
//         The colors of edges in this array are given in the same order as the edges in the "edges" array.
//      4) A qubit "target" in an arbitrary state.
//
// Goal: Implement a marking oracle for function f(x) = 1 if
//       the coloring of the edges of the given graph described by this colors assignment is triangle-free, i.e.,
//       no triangle of edges connecting 3 vertices has all three edges in the same color.
//
// Example: a graph with 3 vertices and 3 edges [(0, 1), (1, 2), (2, 0)] has one triangle.
// The result of applying the operation to state (|001⟩ + |110⟩ + |111⟩)/√3 ⊗ |0⟩ 
// will be 1/√3|001⟩ ⊗ |1⟩ + 1/√3|110⟩ ⊗ |1⟩ + 1/√3|111⟩ ⊗ |0⟩.
// The first two terms describe triangle-free colorings, 
// and the last term describes a coloring where all edges of the triangle have the same color.
//
// In this task you are not allowed to use quantum gates that use more qubits than the number of edges in the graph,
// unless there are 3 or less edges in the graph. For example, if the graph has 4 edges, you can only use 4-qubit gates or less.
// You are guaranteed that in tests that have 4 or more edges in the graph the number of triangles in the graph 
// will be strictly less than the number of edges.
//
// Hint: Make use of helper functions and helper operations, and avoid trying to fit the complete
//       implementation into a single operation - it's not impossible but make your code less readable.
//       GraphColoring kata has an example of implementing oracles for a similar task.
//
// Hint: Remember that you can examine the inputs and the intermediary results of your computations
//       using Message function for classical values and DumpMachine for quantum states.
//

In [45]:
operation task3 (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl 
{
    X(inputs[0]);
    Controlled X(inputs[0..0], inputs[1]);
    Controlled X(inputs[0..0], inputs[2]);
    Controlled X(inputs[1..2], output);
    Controlled X(inputs[0..0], inputs[1]);
    Controlled X(inputs[0..0], inputs[2]);
    X(inputs[0]);
    
    X(output);
}

task3

In [63]:
function getTriangles(edges : (Int, Int)[]) : (Int, Int, Int)[] 
{
    let n = Length(edges);
    
    mutable triangles = new (Int, Int, Int)[n];
    mutable counter = 0;
    
    for i in 0..n-1 
    {
        let (a1, a2) = edges[i];
        for j in i+1..n-1
        {
            let (b1, b2) = edges[j];
            
            // check if atleast one pair is same
            if (a1 == b1 or a2 == b1 or a1 == b2 or a2 == b2)
            {
                for k in j+1..n-1
                {
                    let (c1, c2) = edges[k];
                    
                    // final check
                    if ((c1 == a2 and c2 == b2) or
                        (c1 == a1 and c2 == b2) or
                        (c1 == a2 and c2 == b1) or
                        (c1 == a1 and c2 == b1) or 
                        (c2 == a2 and c1 == b2) or
                        (c2 == a1 and c1 == b2) or
                        (c2 == a2 and c1 == b1) or
                        (c2 == a1 and c1 == b1))
                    {
                        set triangles w/= counter <- (i, j, k);
                        set counter += 1;
                    }

                }
            }
        }
    }
    
    //Console.Write(triangles);    
    return triangles[0..counter-];
}

getTriangles

In [47]:
operation Task4_TriangleFreeColoringOracle 
(
    V : Int, 
    edges : (Int, Int)[], 
    colorsRegister : Qubit[], 
    target : Qubit
) : Unit is Adj+Ctl 
{
    let triangles = getTriangles(edges);
    let len = Length(triangles);
    let qs = new Qubit[len];
    
    for i in 0..len-1 
    {
        let (x, y, z) = triangles[i];
        task3([colorsRegister[x],colorsRegister[y],colorsRegister[z]], qs[i]);
    }
    
    Controlled X(qs, target);
    
    for i in 0..len-1 
    {
        let (x, y, z) = triangles[i];
        task3([colorsRegister[x],colorsRegister[y],colorsRegister[z]], qs[i]);
    }
}

Task4_TriangleFreeColoringOracle

In [60]:
operation testSanity () : Unit
{
    Message(getTriangles([(1, 2), (2, 3), (1, 3)]));    
}

/snippet_.qs(3,12): error QS6210: The type of the given argument does not match the expected type. Got an argument of type (Int, Int, Int)[], expecting one of type String instead.
